In [21]:
#imports here

import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os
import re
import pandas as pd

### This is where the fun begins

In [ ]:
#initalize all values here:
#capitals as per naming convention for python when you want to assign a constant
#padding needed to not read repetative noise
OFFSET = 6
#height in pixels of each row
HEIGHT = 27
#config for the ocr config arg
CONFIG = "--psm 12 --oem 1"
#collect the year for the files later by regular expression but for the time being, since we already know that the numbers are in the range of 0-9 position, we'll just slice

#set up paths
SOURCE_PATH = "data/unprocessed/debit/"
SAVE_PATH = "data/processed/debit/"
#-1 to remove the "/"
file_names = os.listdir(SOURCE_PATH[:-1])
for file in file_names:
    #create vars to hold each column (we will zip em together later to turn into a dataframe)
    #also reset values
    date = []
    description = []
    charge =[]
    total = []

    #here we will assume that the pdf size is 1700 to 2200 (later we will return to add more adaptability)
    #set up base crop dimensision
    left = 325
    top = 397
    right = 1650
    #bottom will be under constant change per loop, so we'll just initalize it
    bottom = top + HEIGHT

    #come back later to allow for more than one page to be processed
    image = convert_from_path(SOURCE_PATH + file)[1]

    #make shift do-while loop
    #we shall read all the way to the bottom, cleaning will be done in the next steps
    #we could switch it for a forloop sure by dividing the remainging length by the total rows height and just seeing how many times we can run it, but I already wrote this
    while bottom <= 2200:
        #Crop the image to just read row by row
        seg = image.crop((left, top, right, bottom))

        #Lets OCR this thing
        info = pytesseract.image_to_string(seg, config=CONFIG)
        print(info, file_name)
        info = info.rstrip("\n")
        info = info.split("\n\n")

        #some in precosion error handeling
        if len(info) > 3:
            date.append(info[0])
            description.append(info[1])
            charge.append(info[2])
            total.append(info[3])

        #reset the crop values
        top = bottom + OFFSET
        bottom = top + HEIGHT
    
    #turn into a df
    file_name = file[:10]
    df = pd.DataFrame(list(zip(date, description, charge, total)), columns=["date", "description", "charge", "total"])
    #index set to false as it is not needed
    df.to_csv(SAVE_PATH + file_name + ".csv", index=False)





To note over five years of data, only three anomolies showed up and two pages where a second page was given to store more transaction information <br>
solution for anomoly: all three had the same 

In [ ]:
#here we clean the data in all the csvs

#Step one get rid of columns we can't use

Due to time restraints I had to fix some files by hand so that the values I'd use for the model would be at their most accurate

In [ ]:
#here we merge all csv's into one big